In [1]:
# LT 21/09/2022
# extract mutational model parameters from RMC project

import hail as hl
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'regional_missense_constraint',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

ModuleNotFoundError: No module named 'hail'

In [ ]:
# RMC results table
ht = hl.read_table('gs://regional_missense_constraint/model/2.1.1/1/context_obs_exp_annot.ht')

In [ ]:
# filter to ACTA1 canonical transcript
ht = ht.filter(ht.transcript=="ENST00000366684").persist()

In [ ]:
# annotate with the cumulative exp of previous row
ht = ht.annotate(prev_row_cumexp=hl.scan.take(ht.cumulative_exp,1,ordering=-ht.cumulative_exp).first())

# calculate locus-specific expected by taking the difference
ht = ht.annotate(exp = ht.cumulative_exp - hl.coalesce(ht.prev_row_cumexp,0))

# keep what we need and rename observed to obs
ht = ht.select(ht.mutate(obs = ht.observed).obs, ht.exp)

# export
ht.export("ENST00000366684_bp_oe.tsv")